# Nya features
### 1. Kolla vad antal hästar per lopp kan ge
### 2. Kolla vad Bayes target kodning för bana, kusk och häst kan ge

In [1]:
# import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
# Läs in all_data.csv
# select numeric columns
df = pd.read_csv('../all_data.csv')
# df.set_index('datum', inplace=True)
df.columns

Index(['datum', 'avd', 'bana', 'häst', 'kusk', 'streck', 'vodds', 'podds',
       'kr', 'spår', 'dist', 'lopp_dist', 'start', 'ålder', 'kön', 'plac',
       'pris', 'h1_dat', 'h1_kusk', 'h1_bana', 'h1_spår', 'h1_plac', 'h1_pris',
       'h1_odds', 'h1_kmtid', 'h2_dat', 'h2_kusk', 'h2_bana', 'h2_spår',
       'h2_plac', 'h2_pris', 'h2_odds', 'h2_kmtid', 'h3_dat', 'h3_kusk',
       'h3_bana', 'h3_spår', 'h3_plac', 'h3_pris', 'h3_odds', 'h3_kmtid',
       'h4_dat', 'h4_kusk', 'h4_bana', 'h4_spår', 'h4_plac', 'h4_pris',
       'h4_odds', 'h4_kmtid', 'h5_dat', 'h5_kusk', 'h5_bana', 'h5_spår',
       'h5_plac', 'h5_pris', 'h5_odds', 'h5_kmtid', 'h1_dist', 'h2_dist',
       'h3_dist', 'h4_dist', 'h5_dist', 'bins', 'h1_auto', 'h2_auto',
       'h3_auto', 'h4_auto', 'h5_auto', 'h1_perf', 'h2_perf', 'h3_perf',
       'h4_perf', 'h5_perf', 'senast', 'delta1', 'delta2', 'delta3', 'delta4',
       'startnr'],
      dtype='object')

In [3]:
# count number per avdeling
df_count = df.groupby(['datum','avd'])['avd'].count()
df['avd_count'] = None
df['avd_count'] = df.groupby(['datum', 'avd'])['avd'].transform('count')

In [4]:
df['avd_count'].unique()
my_remove_list = ['streck']
sorted(df['avd_count'].unique())

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

# Kolla med catboost

### Några hjälp-funktioner

In [5]:
def split_data(df_,train_from_proc=0,test_proc=0.25):
    df=df_.copy()
    alla_datum = df.datum.unique()
    train_from_datum = alla_datum[ int(len(alla_datum)*train_from_proc)]
    print("train from",train_from_datum)
    X_test=None
    y_test=None
    test_from_datum=alla_datum[-1]
    if test_proc:
        selected_data = alla_datum[ alla_datum >= train_from_datum ]
        test_from_datum = selected_data[ int(len(selected_data)*(1-test_proc)) ]
        print("test from",test_from_datum)
        X_test  = df[df.datum >= test_from_datum]
        y_test  = (X_test.plac==1)*1
        X_test  = X_test.drop('plac',axis=1)
        
    X_train = df[(df.datum >= train_from_datum) & (df.datum < test_from_datum) ]
    y_train = (X_train.plac==1)*1
    
    return X_train.drop('plac', axis=1), X_test, y_train, y_test
### Features som inte används vid träning

def remove_features(df_, remove_mer=[]):
    df = df_.drop(['avd', 'startnr', 'vodds', 'podds', 'bins',
                  'h1_dat', 'h2_dat', 'h3_dat', 'h4_dat', 'h5_dat'], axis=1)
    if remove_mer:
        df = df.drop(remove_mer, axis=1)
    return df


### set up for catboost classifier

In [6]:
# set up catboost classifier model
from catboost import CatBoostClassifier, cv, Pool

X_train, X_test, y_train, y_test = split_data(
    df, train_from_proc=0.3, test_proc=0.25)
X_train = remove_features(X_train, remove_mer=my_remove_list )
X_test = remove_features(X_test, remove_mer=my_remove_list )

cat_features = list(X_train.select_dtypes('object').columns)
X_train[cat_features] = X_train[cat_features].fillna('missing')
X_test[cat_features] = X_test[cat_features].fillna('missing')
print(cat_features)
pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)
X_train.shape, X_test.shape


train from 2017-03-11
test from 2020-07-28
['datum', 'bana', 'häst', 'kusk', 'kön', 'h1_kusk', 'h1_bana', 'h2_kusk', 'h2_bana', 'h3_kusk', 'h3_bana', 'h4_kusk', 'h4_bana', 'h5_kusk', 'h5_bana']


((22850, 68), (7666, 68))

In [14]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22850 entries, 13118 to 35967
Data columns (total 68 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   datum      22850 non-null  object 
 1   bana       22850 non-null  object 
 2   häst       22850 non-null  object 
 3   kusk       22850 non-null  object 
 4   kr         22850 non-null  float64
 5   spår       22761 non-null  float64
 6   dist       22850 non-null  float64
 7   lopp_dist  22850 non-null  float64
 8   start      22850 non-null  int64  
 9   ålder      22850 non-null  int64  
 10  kön        22850 non-null  object 
 11  pris       22850 non-null  float64
 12  h1_kusk    22850 non-null  object 
 13  h1_bana    22850 non-null  object 
 14  h1_spår    21114 non-null  float64
 15  h1_plac    22845 non-null  float64
 16  h1_pris    20909 non-null  float64
 17  h1_odds    22718 non-null  float64
 18  h1_kmtid   22847 non-null  float64
 19  h2_kusk    22850 non-null  object 
 20  h2

In [8]:
# run catboost
cb = CatBoostClassifier(
    iterations=1000,
    # eval_metric='MAE',
    early_stopping_rounds=100,
    random_seed=42,
    verbose=100
)
cb.fit(pool, eval_set=test_pool)

Learning rate set to 0.068636
0:	learn: 0.6272175	test: 0.6276838	best: 0.6276838 (0)	total: 201ms	remaining: 3m 20s
100:	learn: 0.2408297	test: 0.2798247	best: 0.2761176 (49)	total: 8.09s	remaining: 1m 11s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.2761176166
bestIteration = 49

Shrink model to first 50 iterations.


Med streck:  
alla:  
bestTest = 0.2410183541  
bestIteration = 210  

alla 0:or borta:  
bestTest = 0.2412283588  
bestIteration = 67  

Utan streck:  
alla:  
bestTest = 0.2745510195  
bestIteration = 0 

alla 0:or borta:  
bestTest =0.272817442 0.2742631409   
bestIteration = 75  






In [9]:
my_feature_imp =cb.get_feature_importance(prettified=True)
my_remove_list2 = my_feature_imp[my_feature_imp.Importances <= 0]['Feature Id'].to_list()
print(my_remove_list2)
my_remove_list=my_remove_list+my_remove_list2
# remove duplicates
my_remove_list = list(set(my_remove_list))


['start', 'h1_pris', 'h2_kusk', 'h3_kusk', 'h3_plac', 'h3_kmtid', 'h4_plac', 'h5_kusk', 'h5_pris', 'h4_dist', 'h1_auto', 'h3_auto', 'h4_auto', 'h5_auto']


In [122]:
cb.get_feature_importance(prettified=True)

,Feature Id,Importances
0,häst,11.220096
1,pris,9.525040
2,datum,8.302184
3,kr,7.649721
4,spår,5.125639
5,h1_odds,5.035553
6,kusk,4.890557
7,h3_odds,4.772404
8,h1_perf,3.489494
9,h2_odds,2.976799


# Använd Bayes target encoding (mitt eget namn) 
### för Bana, Häst och Kusk (fortsätt senare)

In [184]:
# Bayes function
def compute_Bayes(bana, häst, df_):
    df = df_.copy()
    # p(A|B) = p(B|A) * p(A) / p(B)
    # p(A) häst total performance
    # p(B) Share of bana in df
    # p(B|A) Share of bana where häst is winner
    # p(A|B) probabiity that häst is winner in bana
    # Find häst total performance
    total_performance = df.loc[(df.häst == häst) & (df.plac == 1),['datum','avd','bana','häst','kusk']]
    print(total_performance)
compute_Bayes('AXEVALLA', 'NINJA ZON', df)

# df.loc[(df.häst == 'GOGO GIRL') & (df.plac == 1),'plac']
print(f'Antal ggr NUNCIO har vunnit på SOLVALLA  : {df.loc[(df.bana=="SOLVALLA") & (df.häst=="NUNCIO") & (df.plac==1),"häst"].count()}')
print(f"Antal ggr NUNCIO har sprungit på SOLVALLA: {df.loc[(df.bana == 'SOLVALLA') & (df.häst == 'NUNCIO'), 'häst'].count()}")
print(f"Antal ggr NUNCIO har vunnit totalt       : {df.loc[(df.häst == 'NUNCIO') & (df.plac==1),'häst'].count()}")
print(f"Antal ggr NUNCIO har sprungit totalt     : {df.loc[(df.häst == 'NUNCIO'),'häst'].count()}")


            datum  avd      bana       häst              kusk
43084  2021-12-31    4  AXEVALLA  NINJA ZON  Markus Niklasson
Antal ggr NUNCIO har vunnit på SOLVALLA  : 5
Antal ggr NUNCIO har sprungit på SOLVALLA: 8
Antal ggr NUNCIO har vunnit totalt       : 13
Antal ggr NUNCIO har sprungit totalt     : 18
